**ML model based on Logistic regression to predict purchase possibility through ad with the help of info such as their age, gender and average salary**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import drive
import os
import copy
import math
from os import listdir, makedirs
from os.path import join, exists
from os import listdir
drive.mount('/content/drive')
os.chdir("/content/drive/")
%matplotlib inline
!ls

Mounted at /content/drive
MyDrive  Shareddrives


taking male as 0 and female as 1 and dividing salary by 10,000

In [2]:

data = pd.read_csv('/content/drive/My Drive/logistic_regression/Social_Network_Ads.csv')
labels = [ 'Gender', 'Age', 'EstimatedSalary']
print(len(data))
X_train = data.iloc[:, 1:4]
X_train[labels[2]] = X_train[labels[2]] / 1000
y_train = data.iloc[:, 4]

400


In [3]:
gender = X_train[labels[0]]
i = 0
j = 0
for gend in gender:
  if gend == 'Male':
    X_train[labels[0]][i] = 0
  else:
    X_train[labels[0]][i] = 1
  i += 1


<ipython-input-3-374af2390f5a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[labels[0]][i] = 0
<ipython-input-3-374af2390f5a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[labels[0]][i] = 1


In [4]:
print('First 3 elements of dataset are:\n', X_train[:15])
print("Datatype of X is ",type(X_train))

First 3 elements of dataset are:
    Gender  Age  EstimatedSalary
0       0   19             19.0
1       0   35             20.0
2       1   26             43.0
3       1   27             57.0
4       0   19             76.0
5       0   27             58.0
6       1   27             84.0
7       1   32            150.0
8       0   25             33.0
9       1   35             65.0
10      1   26             80.0
11      1   26             52.0
12      0   20             86.0
13      0   32             18.0
14      0   18             82.0
Datatype of X is  <class 'pandas.core.frame.DataFrame'>


In [5]:
print("First five elements in y_train are:\n", y_train[:15])
print("Type of y_train:",type(y_train))

First five elements in y_train are:
 0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     0
9     0
10    0
11    0
12    0
13    0
14    0
Name: Purchased, dtype: int64
Type of y_train: <class 'pandas.core.series.Series'>


In [6]:
print("Shape of X_train is: "+str(X_train.shape))
print("Shape of y_train is: "+str(y_train.shape))
print("Number of training examples is %d" % len(y_train))

Shape of X_train is: (400, 3)
Shape of y_train is: (400,)
Number of training examples is 400


In [7]:
def sigmoid(z):
  inv = 1/(1+np.exp(-z))
  return inv

In [8]:
value = sigmoid(0)
print(value)

0.5


In [9]:
def compute_gradient(X, y, w, b, *argv): 
    """
    Computes the gradient for logistic regression 
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w. 
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b. 
    """
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.
    i = 0
    for index, row in X_train.iterrows():
        f_wb_i = sigmoid(np.dot(row, w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        dj_db = dj_db + err_i
        for j in range(n):
            dj_dw[j] += err_i * X[labels[j]][i] 
        i += 1
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar

    return dj_db, dj_dw

In [10]:
def compute_cost_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      lambda_ : (scalar, float) Controls amount of regularization
    Returns:
      total_cost : (scalar)     cost 
    """

    m, n = X.shape

    cost_without_reg = compute_cost(X, y, w, b) 

    reg_cost = 0.

    for i in range(n):
        reg_cost += w[i]**2
    reg_cost = ((lambda_/(2 * m)) * reg_cost)

    total_cost = cost_without_reg + reg_cost

    return total_cost

In [11]:
def compute_cost(X, y, w, b, *argv):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns:
      total_cost : (scalar) cost 
    """

    m, n = X.shape
    
    ### START CODE HERE ###
    cost = 0.0
    i = 0
    for index, row in X_train.iterrows():
        z_i = np.dot(row,w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
        i += 1
    total_cost = cost / m
    ### END CODE HERE ### 

    return total_cost

In [12]:
def compute_gradient_reg(X, y, w, b, lambda_ = 1): 
    """
    Computes the gradient for logistic regression with regularization
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      lambda_ : (scalar,float)  regularization constant
    Returns
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b. 
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w. 

    """
    m, n = X.shape
    
    dj_db, dj_dw = compute_gradient(X, y, w, b)
   
    for j in range(n):
        dj_dw[j] += (lambda_/m)*w[j]      
        
    return dj_db, dj_dw

In [13]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (ndarray Shape (m, n) data, m examples by n features
      y :    (ndarray Shape (m,))  target value 
      w_in : (ndarray Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)              Initial value of parameter of the model
      cost_function :              function to compute cost
      gradient_function :          function to compute gradient
      alpha : (float)              Learning rate
      num_iters : (int)            number of iterations to run gradient descent
      lambda_ : (scalar, float)    regularization constant
      
    Returns:
      w : (ndarray Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(X)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

In [33]:
def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape   
    p = np.zeros(m)
   
    ### START CODE HERE ### 
    # Loop over each example
    i = 0
    for index, row in X.iterrows():
        f_wb = sigmoid(np.dot(row,w) + b)
        if(f_wb >= 0.5):
            p[i] = 1
        else:
            p[i] = 0
        i += 1
    ### END CODE HERE ### 
    return p

In [44]:
np.random.seed(1)
initial_w = 0.01 * (np.random.rand(3) - 0.5)
initial_b = -8
print(initial_w)
# Some gradient descent settings
iterations = 1000
alpha = 0.001

w,b, J_history, w_history= gradient_descent(X_train ,y_train, initial_w, initial_b, 
                                   compute_cost, compute_gradient, alpha, iterations, 0.01)

[-0.00082978  0.00220324 -0.00499886]
Iteration    0: Cost     1.76   
Iteration  100: Cost     0.37   
Iteration  200: Cost     0.37   
Iteration  300: Cost     0.37   
Iteration  400: Cost     0.37   
Iteration  500: Cost     0.37   
Iteration  600: Cost     0.37   
Iteration  700: Cost     0.37   
Iteration  800: Cost     0.37   
Iteration  900: Cost     0.37   
Iteration  999: Cost     0.37   


Testing our logistic regression model

In [45]:
X_test = X_train[:20]
print("Weights calculated are:\n")
print(w)
print(b)
print(X_test)
p = predict(X_test,w,b)

Weights calculated are:

[-0.01017778  0.14921535  0.02269733]
-8.008033069397111
   Gender  Age  EstimatedSalary
0       0   19             19.0
1       0   35             20.0
2       1   26             43.0
3       1   27             57.0
4       0   19             76.0
5       0   27             58.0
6       1   27             84.0
7       1   32            150.0
8       0   25             33.0
9       1   35             65.0
10      1   26             80.0
11      1   26             52.0
12      0   20             86.0
13      0   32             18.0
14      0   18             82.0
15      0   29             80.0
16      0   47             25.0
17      0   45             26.0
18      0   46             28.0
19      1   48             29.0


calculating accuracy of our model

In [46]:
n = len(p)
j = 0
true = y_train[:20]
for i in range(n):
  if p[i] == true[i]:
    j += 1
pred = (j/n)*100.00

In [47]:
print(pred)

80.0
